In [1]:
# 라이브러리 import
# =============================================
import os
import time
import re
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# Selenium 드라이버 생성
# =============================================
options = Options()
# options.add_argument("--headless")  # 안정화 후 사용
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

In [4]:
# place_id 로드
# =============================================
FILES = {
    "역전": r"C:\Users\user\Desktop\데이터분석\역전할머니_정제완료.csv",
    "금별": r"C:\Users\user\Desktop\데이터분석\금별맥주_정제완료.csv"
}

PLACE_IDS = {}

for brand, path in FILES.items():
    df = pd.read_csv(path)
    PLACE_IDS[brand] = (
        df["place_id"]
        .dropna()
        .astype(int)
        .unique()
        .tolist()
    )
    print(f"[{brand}] place_id 개수: {len(PLACE_IDS[brand])}")

[역전] place_id 개수: 327
[금별] place_id 개수: 103


In [5]:
# 출력 / 체크포인트 경로
# =============================================
OUTPUT_DIR = r"C:\Users\user\Desktop\데이터분석\출력"
os.makedirs(OUTPUT_DIR, exist_ok=True)

PATHS = {
    "역전": {
        "checkpoint": os.path.join(OUTPUT_DIR, "역전_역과의거리_checkpoint.csv"),
        "final": os.path.join(OUTPUT_DIR, "역전_역과의거리.csv")
    },
    "금별": {
        "checkpoint": os.path.join(OUTPUT_DIR, "금별_역과의거리_checkpoint.csv"),
        "final": os.path.join(OUTPUT_DIR, "금별_역과의거리.csv")
    }
}

In [9]:
# 역과의 거리 추출 (차단 포함)
# =============================================
def extract_exit_info(place_id):
    url = f"https://pcmap.place.naver.com/place/{place_id}/home"
    driver.get(url)
    time.sleep(5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    page_text = soup.get_text(" ")

    # 네이버 차단 감지
    if "접속이 차단" in page_text or "자동입력" in page_text:
        raise Exception("네이버 차단 감지")

    # 1️⃣ 역·출구·거리 정보가 들어있는 div 직접 탐색
    exit_div = soup.select_one("div.nZapA")
    if not exit_div:
        return None

    # 2️⃣ 텍스트 정제
    exit_text = exit_div.get_text(" ", strip=True)

    # 예: "지축역 1번 출구에서 203 m"
    pattern = re.compile(r'([가-힣]+역\s*\d+번\s*출구에서\s*\d+\s*m)')
    match = pattern.search(exit_text)

    return match.group(1) if match else None


In [10]:
# 메인 크롤링
# =============================================
for brand in ["역전", "금별"]:
    print("\n====================================")
    print(f"[{brand}] 역과의 거리 크롤링 시작")
    print("====================================")

    place_ids = PLACE_IDS[brand]
    checkpoint_path = PATHS[brand]["checkpoint"]
    final_path = PATHS[brand]["final"]

    if os.path.exists(checkpoint_path):
        results = pd.read_csv(checkpoint_path).to_dict("records")
        processed_ids = set(r["place_id"] for r in results)
        print(f"[{brand}] 체크포인트 로드: {len(processed_ids)}")
    else:
        results = []
        processed_ids = set()
        print(f"[{brand}] 신규 크롤링")

    print(f"[{brand}] 이번 대상: {len(place_ids)}")

    for idx, pid in enumerate(place_ids, 1):
        if pid in processed_ids:
            continue

        print(f"[{brand}] [{idx}/{len(place_ids)}] place_id={pid}", end=" → ")

        try:
            exit_info = extract_exit_info(pid)
            print(exit_info if exit_info else "정보 없음")

            results.append({
                "place_id": pid,
                "exit_info": exit_info,
                "status": "success" if exit_info else "no_data"
            })

        except Exception as e:
            print(f"오류: {e}")
            results.append({
                "place_id": pid,
                "exit_info": None,
                "status": "error",
                "error": str(e)
            })

        if len(results) % 5 == 0:
            pd.DataFrame(results).to_csv(
                checkpoint_path,
                index=False,
                encoding="utf-8-sig"
            )
            print(f"[{brand}] 💾 체크포인트 저장 ({len(results)})")
            time.sleep(120)

    pd.DataFrame(results).to_csv(
        final_path,
        index=False,
        encoding="utf-8-sig"
    )

    print(f"[{brand}] 1차 크롤링 완료")


[역전] 역과의 거리 크롤링 시작
[역전] 신규 크롤링
[역전] 이번 대상: 327
[역전] [1/327] place_id=1868359886 → 정보 없음
[역전] [2/327] place_id=1323812622 → 정보 없음
[역전] [3/327] place_id=1311695977 → 신풍역 3번 출구에서 200 m
[역전] [4/327] place_id=1650551571 → 상봉역 3번 출구에서 315 m
[역전] [5/327] place_id=1404198758 → 오류동역 3번 출구에서 377 m
[역전] 💾 체크포인트 저장 (5)
[역전] [6/327] place_id=1054587722 → 방배역 3번 출구에서 84 m
[역전] [7/327] place_id=1573115649 → 회기역 1번 출구에서 596 m
[역전] [8/327] place_id=1407805860 → 정보 없음
[역전] [9/327] place_id=1420076532 → 화곡역 7번 출구에서 439 m
[역전] [10/327] place_id=1337289813 → 신대방삼거리역 2번 출구에서 91 m
[역전] 💾 체크포인트 저장 (10)
[역전] [11/327] place_id=1059948798 → 상월곡역 1번 출구에서 320 m
[역전] [12/327] place_id=1199375408 → 성수역 1번 출구에서 54 m
[역전] [13/327] place_id=1679915153 → 영등포구청역 4번 출구에서 118 m
[역전] [14/327] place_id=1125792665 → 상수역 1번 출구에서 454 m
[역전] [15/327] place_id=1424284773 → 서울대벤처타운역 2번 출구에서 493 m
[역전] 💾 체크포인트 저장 (15)
[역전] [16/327] place_id=1093149808 → 가락시장역 3번 출구에서 556 m
[역전] [17/327] place_id=1383332046 → 사가정역 1번 출구에서 152 m
[역전

In [11]:
# 실패 / 정보없음 재크롤링 후 덮어쓰기
# =============================================
for brand in ["역전", "금별"]:
    final_path = PATHS[brand]["final"]
    df = pd.read_csv(final_path)

    retry_df = df[df["status"].isin(["error", "no_data"])]

    if retry_df.empty:
        print(f"[{brand}] 재크롤링 대상 없음")
        continue

    print(f"[{brand}] 재크롤링 대상: {len(retry_df)}")

    for idx, row in retry_df.iterrows():
        pid = row["place_id"]
        print(f"[{brand}] 재크롤링 place_id={pid}", end=" → ")

        try:
            exit_info = extract_exit_info(pid)
            print(exit_info if exit_info else "여전히 없음")

            df.loc[df["place_id"] == pid, ["exit_info", "status"]] = [
                exit_info,
                "success" if exit_info else "no_data"
            ]

        except Exception:
            print("실패 유지")

        time.sleep(15)

    df.to_csv(final_path, index=False, encoding="utf-8-sig")
    print(f"[{brand}] 재크롤링 반영 완료")

[역전] 재크롤링 대상: 114
[역전] 재크롤링 place_id=1868359886 → 실패 유지
[역전] 재크롤링 place_id=1323812622 → 실패 유지
[역전] 재크롤링 place_id=1407805860 → 실패 유지
[역전] 재크롤링 place_id=1092241695 → 실패 유지
[역전] 재크롤링 place_id=1649791385 → 실패 유지
[역전] 재크롤링 place_id=1691525060 → 실패 유지
[역전] 재크롤링 place_id=1449854694 → 실패 유지
[역전] 재크롤링 place_id=1708276745 → 실패 유지
[역전] 재크롤링 place_id=1939413310 → 실패 유지
[역전] 재크롤링 place_id=1528559468 → 실패 유지
[역전] 재크롤링 place_id=1115085909 → 실패 유지
[역전] 재크롤링 place_id=1397283887 → 실패 유지
[역전] 재크롤링 place_id=1358873748 → 실패 유지
[역전] 재크롤링 place_id=1027060589 → 실패 유지
[역전] 재크롤링 place_id=1078478269 → 실패 유지
[역전] 재크롤링 place_id=2073433504 → 실패 유지
[역전] 재크롤링 place_id=1925758063 → 실패 유지
[역전] 재크롤링 place_id=2004339615 → 실패 유지
[역전] 재크롤링 place_id=1500937003 → 실패 유지
[역전] 재크롤링 place_id=1375891395 → 실패 유지
[역전] 재크롤링 place_id=1202988463 → 실패 유지
[역전] 재크롤링 place_id=1081200889 → 실패 유지
[역전] 재크롤링 place_id=1269638080 → 실패 유지
[역전] 재크롤링 place_id=1121119852 → 실패 유지
[역전] 재크롤링 place_id=1714605319 → 실패 유지
[역전] 재크롤링 place_id=1543726258 → 

In [12]:
# 종료
# =============================================
driver.quit()
print("드라이버 종료")

드라이버 종료
